In [3]:
# load image from aws s3 bucket
import boto3
import tensorflow as tf
from PIL import Image
import numpy as np
s3 = boto3.resource('s3')
dataset = s3.Bucket('dalle2images')
image_paths = []

bucket_name = 'dalle2images'
for obj in dataset.objects.all():
    if obj.key.endswith('.png') and obj.key.startswith('real'):
        image_paths.append(obj.key)

    elif obj.key.endswith('.png') and obj.key.startswith('fake'):
        image_paths.append(obj.key)


print("Number of images: %d" % len(image_paths))
print("Number of labels: %d" % len(labels))

Number of images: 42878
Number of labels: 42878


In [79]:
bucket_name = 'dalle2images'
image = []
labels = []
for i in range(2,len(image_paths), 8):
    # read image from s3 bucket
    img = s3.Object(bucket_name, image_paths[i])
    img = Image.open(img.get()['Body']).resize([128,128])
    # convert image to numpy array
    img = np.array(img).astype(np.float32)
    # img = tf.convert_to_tensor(img, dtype=tf.float32)
    # add image to list
    image.append(img/255)
    if i < 21439:
      labels.append(0)
    else:
      labels.append(1)

print("Number of images: %d" % len(image))

Number of images: 16078


In [80]:
len(image)

16078

In [81]:
len(labels)

16078

In [35]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [82]:
#make labels into numpy array
alllabels = np.array(labels)

In [83]:
# reshape image
allimage = np.array(image).reshape(-1, 128, 128, 3)

In [84]:
#split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(allimage, alllabels, test_size=0.2, random_state=42)

# MobileV2 model

In [53]:
# create model using gpu
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.regularizers import l2


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add the data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

# Define the regularization strength
reg_strength = 0.001

# Create a new model using the base model and adding custom top layers
mobilemodel = tf.keras.models.Sequential([
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(reg_strength)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
mobilemodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 0s 0us/step


In [85]:
# Train the model for 10 epochs using the train dataset and avoid crashing
epochs = 10

mobilemodel.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
402/402 [==============================] - 6s 14ms/step - loss: 0.5709 - accuracy: 0.7404 - val_loss: 0.5264 - val_accuracy: 0.7814
Epoch 2/10
402/402 [==============================] - 5s 12ms/step - loss: 0.5703 - accuracy: 0.7407 - val_loss: 0.5227 - val_accuracy: 0.7836
Epoch 3/10
402/402 [==============================] - 5s 12ms/step - loss: 0.5666 - accuracy: 0.7453 - val_loss: 0.5266 - val_accuracy: 0.7637
Epoch 4/10
402/402 [==============================] - 5s 12ms/step - loss: 0.5644 - accuracy: 0.7470 - val_loss: 0.4868 - val_accuracy: 0.8038
Epoch 5/10
402/402 [==============================] - 5s 12ms/step - loss: 0.5654 - accuracy: 0.7441 - val_loss: 0.4864 - val_accuracy: 0.7998
Epoch 6/10
402/402 [==============================] - 5s 12ms/step - loss: 0.5684 - accuracy: 0.7395 - val_loss: 0.5080 - val_accuracy: 0.7870
Epoch 7/10
402/402 [==============================] - 5s 12ms/step - loss: 0.5648 - accuracy: 0.7408 - val_loss: 0.4848 - val_accuracy: 0.7970

In [86]:
pickle.dump(mobilemodel, open('mobilemodel3.pkl', 'wb'))

In [87]:
# Evaluate the model
loss, accuracy = mobilemodel.evaluate(X_test, y_test)
print("Test accuracy: {:.2f}%".format(accuracy * 100))

101/101 [==============================] - 1s 8ms/step - loss: 0.4857 - accuracy: 0.7951
Test accuracy: 79.51%


In [56]:
# Evaluate the model on the less dataset
loss, accuracy = mobilemodel.evaluate(X_test, y_test)
print("Test accuracy: {:.2f}%".format(accuracy * 100))

34/34 [==============================] - 0s 8ms/step - loss: 0.5676 - accuracy: 0.7901
Test accuracy: 79.01%
